# Check publication
When publishing a service there are several things to check. 
1. the deletion protection should be enabled
2. check the shared options (public for tile services, whitelisted for feature services)
3. check the visibility of the service (zoom levels)
4. check the spatial reference

In [1]:
import re
def create_strict_reg_exp(to_search):
    try:
        reg_exp = f"^{to_search}$"
        #logging.info(f"regular expression is: {reg_exp}")
    except:
        print("There was a problem with the string.")
    return reg_exp

In [2]:
def findItemGetID(csvName, gis, item_type = "Feature Layer"):
    try:
        searched_item = gis.content.search(csvName, item_type = item_type)
        for i in searched_item:
            reg_exp = create_strict_reg_exp(csvName)
            if re.search(reg_exp, i.title)!= None:    
                #logging.info(f"{csvName} has the id: {i.id}")
                return i.id
    except:
        print("There was a problem finding the item")

In [3]:
import arcgis
from arcgis.gis import GIS
import requests
import pandas as pd

In [4]:
gis = GIS(profile = "eowilson")

# search by name of service

In [5]:
searched_item = gis.content.search("Putative_PA_network", item_type = "Map Image Layer")

In [30]:
item_list = list()
for s in searched_item:
    print(s.name)
    item_list.append(s.name)
del item_list[3]
print(item_list)

Putative_PA_network_PR_rept
Putative_PA_network_PR_bird
Putative_PA_network_PR_all
putative_pa_ranked_global
Putative_PA_network_PR_mamm
Putative_PA_network_PR_amph
['Putative_PA_network_PR_rept', 'Putative_PA_network_PR_bird', 'Putative_PA_network_PR_all', 'Putative_PA_network_PR_mamm', 'Putative_PA_network_PR_amph']


In [45]:
for item in item_list:
    a = findItemGetID(item, gis, item_type = "Map Image Layer")
    g_item = gis.content.get(a)
    request = requests.get(g_item.url + '?f=pjson&token={}'.format(gis._con.token))
    service_layer_json =  request.json()
    print(f"name: {g_item.name}")
    print(f"name: {g_item.owner}")
    print(f"url: {g_item.url}")
    print(f"delete protection enabled: {g_item.protected}")
    print(f"share level: {g_item.access}")
    print(f"maxScale: {service_layer_json['maxScale']}")
    print(f"minScale: {service_layer_json['minScale']}")
    print(f"spatial reference: {service_layer_json['spatialReference']}")
    #print(f"levels of tiles: {len(service_layer_json['tileInfo']['lods'])}") # this is not useful for reviewing. 
    print("-------")

name: Putative_PA_network_PR_rept
name: greta.carrete_eowilson
url: https://tiles.arcgis.com/tiles/IkktFdUAcY3WrH25/arcgis/rest/services/Putative_PA_network_PR_rept/MapServer
delete protection enabled: True
share level: public
maxScale: 0.0
minScale: 591657527.591555
spatial reference: {'wkid': 102100, 'latestWkid': 3857}
-------
name: Putative_PA_network_PR_bird
name: greta.carrete_eowilson
url: https://tiles.arcgis.com/tiles/IkktFdUAcY3WrH25/arcgis/rest/services/Putative_PA_network_PR_bird/MapServer
delete protection enabled: True
share level: public
maxScale: 0.0
minScale: 591657527.591555
spatial reference: {'wkid': 102100, 'latestWkid': 3857}
-------
name: Putative_PA_network_PR_all
name: greta.carrete_eowilson
url: https://tiles.arcgis.com/tiles/IkktFdUAcY3WrH25/arcgis/rest/services/Putative_PA_network_PR_all/MapServer
delete protection enabled: True
share level: public
maxScale: 0.0
minScale: 591657527.591555
spatial reference: {'wkid': 102100, 'latestWkid': 3857}
-------
name: 

# Searching by urls in csv

In [5]:
csv_file = "/Users/gretacvega/Documents/GitHub/he-scratchfolder/items_to_check.csv"

In [6]:
item_df = pd.read_csv(csv_file, header=None)

In [7]:
item_df.shape

(9, 1)

In [8]:
url_list = item_df[0].tolist()

In [10]:
url_list

['https://eowilson.maps.arcgis.com/home/item.html?id=a82e287b45a64fe680ea45c61f609204',
 'https://eowilson.maps.arcgis.com/home/item.html?id=b510034983154975bb1d8cb362eed875',
 'https://eowilson.maps.arcgis.com/home/item.html?id=b92cf2914cba4b97bb28ae4328484a6d',
 'https://eowilson.maps.arcgis.com/home/item.html?id=7bc7a9e9020a4411b2753a2f6ec3b735',
 'https://eowilson.maps.arcgis.com/home/item.html?id=698574dd82954165a23123bcee005fb0',
 'https://eowilson.maps.arcgis.com/home/item.html?id=9c48069afd1b454dbc45fc7fbd16ffdc',
 'https://eowilson.maps.arcgis.com/home/item.html?id=10fb899e16f24517a35f264283306e7f',
 'https://eowilson.maps.arcgis.com/home/item.html?id=c279e7dfd2bd4b57bdfc1d547f51bdfd',
 'https://eowilson.maps.arcgis.com/home/item.html?id=d0a129782ee540358ba55b7c514e2bf2']

In [9]:
for item in url_list:
    a = item[-32:]
    g_item = gis.content.get(a)
    request = requests.get(g_item.url + '?f=pjson&token={}'.format(gis._con.token))
    service_layer_json =  request.json()
    print(f"name: {g_item.name}")
    print(f"name: {g_item.owner}")
    print(f"url: {g_item.url}")
    print(f"delete protection enabled: {g_item.protected}")
    if g_item.protected != True:
        print(f"check settings and enable prtection deletion: {item}")
    print(f"share level: {g_item.access}")
    if g_item.access!="public":
        print(f"update sharing levels to public: {item}")
    print(f"maxScale: {service_layer_json['maxScale']}")
    print(f"minScale: {service_layer_json['minScale']}")
    if service_layer_json['minScale']!=591657527.591555:
        print(f"check settings and visibility: {item}")
    print(f"spatial reference: {service_layer_json['spatialReference']}")
    #print(f"levels of tiles: {len(service_layer_json['tileInfo']['lods'])}") # this is not useful for reviewing. 
    print("-------")

name: rarity_marine_mammals_and_fish
name: greta.carrete_eowilson
url: https://tiles.arcgis.com/tiles/IkktFdUAcY3WrH25/arcgis/rest/services/rarity_marine_mammals_and_fish/MapServer
delete protection enabled: True
share level: public
maxScale: 0.0
minScale: 591657527.591555
spatial reference: {'wkid': 102100, 'latestWkid': 3857}
-------
name: richness_marine_mammals_and_fish
name: greta.carrete_eowilson
url: https://tiles.arcgis.com/tiles/IkktFdUAcY3WrH25/arcgis/rest/services/richness_marine_mammals_and_fish/MapServer
delete protection enabled: True
share level: public
maxScale: 0.0
minScale: 591657527.591555
spatial reference: {'wkid': 102100, 'latestWkid': 3857}
-------
name: Global_marine_mammals_prioritisation_TL
name: greta.carrete_eowilson
url: https://tiles.arcgis.com/tiles/IkktFdUAcY3WrH25/arcgis/rest/services/Global_marine_mammals_prioritisation_TL/MapServer
delete protection enabled: True
share level: public
maxScale: 1155581.108577
minScale: 591657527.591555
spatial reference

# Defining function

In [32]:
import requests
import json
def service_qa_tiles(item_name, gis, item_type = "Map Image Layer"):
    a = findItemGetID(item_name, gis, item_type)
    g_item = gis.content.get(a)
    request = requests.get(g_item.url + '?f=pjson&token={}'.format(gis._con.token))
    service_layer_json =  request.json()
    print(f"name: {g_item.name}")
    print(f"url: {g_item.url}")
    print(f"delete protection enabled: {g_item.protected}")
    print(f"share level: {g_item.access}")
    print(f"maxScale: {service_layer_json['maxScale']}")
    print(f"minScale: {service_layer_json['minScale']}")
    print(f"spatial reference: {service_layer_json['spatialReference']}")
    #print(f"levels of tiles: {len(service_layer_json['tileInfo']['lods'])}") # this is not useful for reviewing. 
    print("-------")

In [33]:
service_qa_tiles("Prioritisation_for_national_report_cards", gis)

name: Prioritisation_for_national_report_cards
url: https://tiles.arcgis.com/tiles/IkktFdUAcY3WrH25/arcgis/rest/services/Prioritisation_for_national_report_cards/MapServer
delete protection enabled: True
share level: public
maxScale: 0.0
minScale: 591657527.591555
spatial reference: {'wkid': 102100, 'latestWkid': 3857}
-------


In [34]:
searched_item = gis.content.search("title:Global Species Patterns", item_type = "Map Image Layer", max_items=50)

In [41]:
import requests
import json
def service_url(item_name, gis, item_type = "Map Image Layer"):
    try:
        print(item_name)
        a = findItemGetID(item_name, gis, item_type)
        g_item = gis.content.get(a)

        request = requests.get(g_item.url + '?f=pjson&token={}'.format(gis._con.token))
        service_layer_json =  request.json()
        print(f"name: {g_item.title}")
        print(f"name: {g_item.name}")
        print(f"name: {g_item.owner}")
        print(f"url: {g_item.url}")
        print(f"delete protection enabled: {g_item.protected}")
        print(f"share level: {g_item.access}")
        print(f"spatial reference: {g_item.spatialReference}")
        #print(f"maxScale: {service_layer_json['maxScale']}")
        #print(f"minScale: {service_layer_json['minScale']}")
        #print(f"levels of tiles: {len(service_layer_json['tileInfo']['lods'])}") # this is not useful for reviewing. 
        print("-------")
    except:
        print("failed")
        print("-------")

In [42]:
for i in searched_item:
    
    service_url(i.title, gis)

Global Terrestrial Species Richness Patterns for Select Taxa
name: Global Terrestrial Species Richness Patterns for Select Taxa
name: all_richness
name: wendy_eowilson
url: https://tiles.arcgis.com/tiles/IkktFdUAcY3WrH25/arcgis/rest/services/all_richness/MapServer
delete protection enabled: False
share level: shared
spatial reference: 4326
-------
Global Species Richness Patterns for Cacti [DEPRECATED]
failed
-------
Global Species Rarity Patterns for Cacti [DEPRECATED]
failed
-------
Global Species Richness Patterns for Conifers [DEPRECATED]
failed
-------
Global Species Richness Patterns for Conifers [DEPRECATED]
failed
-------
Global Marine Species Rarity (55km)
failed
-------
Global Reptile Species Rarity (55km)
failed
-------
Global Conifer Species Rarity (55km)
failed
-------
Conservation Prioritization for Global Marine Fish Species (55km)
failed
-------
Global Bird Species Rarity (55km)
failed
-------
Global Terrestrial Species Richness (55km)
failed
-------
Global Amphibian Sp